# Chương 7: Phân tích Nhân tố (EFA, CFA) và Mô hình Cấu trúc (SEM)

**Mục tiêu:** Xây dựng và kiểm định các thang đo cho các khái niệm trừu tượng (Biến tiềm ẩn - Latent Variables), từ đó kiểm định các mô hình lý thuyết phức tạp.

---

## 7.1. Dẫn nhập: Đo lường cái "Vô hình"

### Case Study: Đo lường "Chất lượng dịch vụ"
Bạn không thể cầm thước dây ra đo "Chất lượng dịch vụ" của một ngân hàng. Bạn chỉ có thể đo nó gián tiếp qua các câu hỏi khảo sát (thang đo Likert 1-5):
* Q1: Nhân viên nhiệt tình.
* Q2: Nhân viên hiểu biết.
* Q3: Giao dịch nhanh chóng.
* Q4: ATM luôn hoạt động.

**Vấn đề:** Làm sao biết Q1 và Q2 cùng đo lường một khái niệm (ví dụ: "Năng lực nhân viên"), còn Q3 và Q4 đo lường cái khác (ví dụ: "Cơ sở vật chất")?

**Phân tích Nhân tố (Factor Analysis)** sẽ giúp gom các câu hỏi này lại thành các nhóm biến tiềm ẩn (Factors).

## 7.2. Phân tích Nhân tố Khám phá (EFA)

### 7.2.1. EFA khác gì PCA?
* **PCA (Chương 6):** Rút gọn dữ liệu thuần túy toán học. Giữ lại tổng phương sai càng nhiều càng tốt.
* **EFA (Chương 7):** Tìm ra cấu trúc ngầm (Latent structure). Giả định rằng biến quan sát là sự thể hiện của biến tiềm ẩn.

### 7.2.2. Các chỉ số quan trọng
1.  **Hệ số tải nhân tố (Factor Loading):** Tương quan giữa câu hỏi và nhân tố. Yêu cầu > 0.5.
2.  **KMO & Bartlett Test:** Kiểm tra xem dữ liệu có đủ điều kiện để chạy nhân tố không (KMO > 0.6).
3.  **Cronbach's Alpha:** Kiểm định độ tin cậy của thang đo (Các câu hỏi trong cùng 1 nhóm có nhất quán không). Yêu cầu > 0.7.

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Lưu ý: Để chạy EFA cần cài thư viện factor_analyzer
# !pip install factor_analyzer
from factor_analyzer import FactorAnalyzer
from factor_analyzer.factor_analyzer import calculate_kmo, calculate_bartlett_sphericity

# --- GIẢ LẬP DỮ LIỆU KHẢO SÁT ---
np.random.seed(123)
n_respondents = 300

# Giả lập 2 Nhân tố: 
# F1: Tin cậy (Trust) - đo bằng T1, T2, T3
# F2: Hài lòng (Satisfaction) - đo bằng S1, S2, S3

F1_Trust = np.random.normal(3.5, 0.8, n_respondents) # Biến tiềm ẩn Trust
F2_Sat = 0.6 * F1_Trust + np.random.normal(0, 0.5, n_respondents) # Hài lòng chịu ảnh hưởng bởi Trust

# Tạo biến quan sát từ biến tiềm ẩn (cộng thêm nhiễu)
data_survey = pd.DataFrame({
    'T1': F1_Trust + np.random.normal(0, 0.4, n_respondents),
    'T2': F1_Trust + np.random.normal(0, 0.5, n_respondents),
    'T3': 0.9*F1_Trust + np.random.normal(0, 0.4, n_respondents),
    
    'S1': F2_Sat + np.random.normal(0, 0.4, n_respondents),
    'S2': 0.8*F2_Sat + np.random.normal(0, 0.5, n_respondents),
    'S3': F2_Sat + np.random.normal(0, 0.6, n_respondents),
    
    # Biến rác (Noise) để xem EFA có loại được không
    'Noise1': np.random.normal(3, 1, n_respondents)
})

# Làm tròn về thang đo 1-5 (Likert)
data_survey = data_survey.round().clip(1, 5)

print("Dữ liệu khảo sát giả lập (5 dòng đầu):")
print(data_survey.head())

Dữ liệu khảo sát giả lập (5 dòng đầu):
    T1   T2   T3   S1   S2   S3  Noise1
0  3.0  3.0  3.0  3.0  1.0  2.0     2.0
1  4.0  5.0  4.0  3.0  1.0  2.0     4.0
2  4.0  5.0  4.0  2.0  1.0  2.0     5.0
3  2.0  2.0  2.0  2.0  1.0  2.0     3.0
4  4.0  3.0  2.0  1.0  2.0  3.0     2.0


In [5]:
# --- BƯỚC 1: KIỂM ĐỊNH ĐIỀU KIỆN (KMO & BARTLETT) ---
kmo_all, kmo_model = calculate_kmo(data_survey)
chi_square_value, p_value = calculate_bartlett_sphericity(data_survey)

print(f"KMO Index: {kmo_model:.3f} (Yêu cầu > 0.6)")
print(f"Bartlett P-value: {p_value:.4f} (Yêu cầu < 0.05)")

# --- BƯỚC 2: CHẠY EFA ---
# Sử dụng phép xoay 'promax' (cho phép các nhân tố tương quan với nhau - thực tế hơn varimax)
fa = FactorAnalyzer(n_factors=2, rotation='promax')
fa.fit(data_survey)

# Xem hệ số tải (Loadings)
loadings = pd.DataFrame(fa.loadings_, index=data_survey.columns, columns=['Factor1', 'Factor2'])
print("\nMa trận Hệ số tải (Factor Loadings) - Đã xoay:")
print(loadings.round(3))

KMO Index: 0.823 (Yêu cầu > 0.6)
Bartlett P-value: 0.0000 (Yêu cầu < 0.05)

Ma trận Hệ số tải (Factor Loadings) - Đã xoay:
        Factor1  Factor2
T1        0.263    0.712
T2        0.229    0.699
T3        0.323    0.630
S1        0.777   -0.025
S2        0.672   -0.052
S3        0.695    0.021
Noise1   -0.050    0.102


**Diễn giải kết quả:**
* Các biến T1, T2, T3 sẽ có hệ số tải cao ở cùng 1 cột (Nhóm Tin cậy).
* Các biến S1, S2, S3 sẽ cao ở cột còn lại (Nhóm Hài lòng).
* Biến `Noise1` có hệ số tải thấp ở cả 2 cột (< 0.4) $\rightarrow$ Loại bỏ biến rác này khỏi thang đo.

## 7.3. Phân tích Nhân tố Khẳng định (CFA) và SEM

### 7.3.1. Sự khác biệt EFA vs. CFA
* **EFA:** "Dữ liệu ơi, hãy cho tôi biết có bao nhiêu nhóm?" (Dò đường).
* **CFA:** "Tôi tin rằng có 2 nhóm, hãy kiểm tra xem dữ liệu có khớp với niềm tin đó không?" (Kiểm chứng).

### 7.3.2. Mô hình Cấu trúc Tuyến tính (SEM)
SEM kết hợp CFA (Mô hình đo lường) và Path Analysis (Mô hình cấu trúc/Hồi quy).
Nó cho phép kiểm định một chuỗi quan hệ: **A tác động lên B, và B tác động lên C** cùng một lúc.

In [6]:
# Để chạy SEM trong Python, ta dùng thư viện 'semopy'
# !pip install semopy
import semopy

# --- BƯỚC 3: ĐỊNH NGHĨA MÔ HÌNH LÝ THUYẾT ---
# Cú pháp của semopy:
# =~ : Định nghĩa biến tiềm ẩn (Measurement part)
# ~  : Định nghĩa hồi quy (Structural part)

model_desc = """
# Measurement Model (CFA)
Trust =~ T1 + T2 + T3
Satisfaction =~ S1 + S2 + S3

# Structural Model (Regression)
Satisfaction ~ Trust
"""

# --- BƯỚC 4: HUẤN LUYỆN MÔ HÌNH SEM ---
model = semopy.Model(model_desc)
results = model.fit(data_survey)

# --- BƯỚC 5: XEM KẾT QUẢ ---
estimates = model.inspect()
print("Kết quả ước lượng mô hình SEM:")
print(estimates[estimates['op'].isin(['~', '=~'])])

# Đánh giá độ phù hợp mô hình (Model Fit)
stats_fit = semopy.calc_stats(model)
print("\nChỉ số độ phù hợp (Model Fit Indices):")
print(stats_fit[['CFI', 'RMSEA', 'GFI']].T)

Kết quả ước lượng mô hình SEM:
           lval op          rval  Estimate  Std. Err    z-value p-value
0  Satisfaction  ~         Trust  0.480547  0.052102    9.22328     0.0
1            T1  ~         Trust  1.000000         -          -       -
2            T2  ~         Trust  0.954152  0.060363  15.806985     0.0
3            T3  ~         Trust  0.914842  0.055736  16.413788     0.0
4            S1  ~  Satisfaction  1.000000         -          -       -
5            S2  ~  Satisfaction  0.738155   0.07858   9.393724     0.0
6            S3  ~  Satisfaction  1.023307  0.100928  10.138982     0.0

Chỉ số độ phù hợp (Model Fit Indices):
          Value
CFI    1.001064
RMSEA  0.000000
GFI    0.990727


### 7.3.3. Đọc chỉ số Model Fit (Rất quan trọng)
Khi báo cáo kết quả SEM/CFA, sinh viên phải trình bày các chỉ số này:
1.  **CFI (Comparative Fit Index):** Yêu cầu > 0.9 (Càng gần 1 càng tốt).
2.  **RMSEA (Root Mean Square Error of Approximation):** Sai số. Yêu cầu < 0.08 (Càng nhỏ càng tốt).
3.  **P-value (Estimate):** Kiểm tra xem các mối quan hệ (mũi tên) có ý nghĩa thống kê không.

---

## 7.4. Tổng kết Chương 7

1.  **Quy trình chuẩn:** Cronbach's Alpha $\rightarrow$ EFA $\rightarrow$ CFA $\rightarrow$ SEM.
2.  **EFA** dùng để lọc thang đo rác.
3.  **SEM** dùng để khẳng định mô hình lý thuyết.

### Bài tập thực hành
Lấy bộ dữ liệu mẫu về **Sự gắn kết nhân viên** (Employee Engagement).
* Chạy EFA để nhóm các câu hỏi thành 3 nhóm: Lương thưởng, Môi trường, Lãnh đạo.
* Dùng SEM để kiểm định xem yếu tố nào tác động mạnh nhất đến "Ý định nghỉ việc".